In [2]:
from ultralytics import YOLO
MODEL_PATH = "/Users/kwonsebin/Documents/khu/4-1/capstone/runs/detect/train13/weights/best.pt"
MODEL_PATH1 = "/Users/kwonsebin/Documents/khu/4-1/capstone/runs/detect/train82/weights/best.pt"

detection_model = YOLO(MODEL_PATH)
detection_model1 = YOLO(MODEL_PATH1)


In [17]:
import subprocess
from flask import Flask, request, jsonify, render_template, send_from_directory
from flask_socketio import SocketIO, emit
from ultralytics import YOLO
import os, cv2
import numpy as np
from sklearn.cluster import HDBSCAN
from collections import Counter
from math import sqrt
from sklearn.linear_model import LinearRegression
import mediapipe as mp
from numpy import dot
from numpy.linalg import norm
from IsTriangle import is_triangle_line_in_box
from Average import average
from cos import cosx
# from findFoot import find_foot

video_path = "uploads/8.mp4"

detection_Ball_Glove = detection_model.track(video_path, save=False)
detection_results3 = detection_model1.track(video_path, save=False)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/341) /Users/kwonsebin/Documents/khu/4-1/capstone/server/uploads/8.mp4: 384x640 1 Baseball_ball, 71.9ms
video 1/1 (frame 2/341) /Users/kwonsebin/Documents/khu/4-1/capstone/server/uploads/8.mp4: 384x640 1 Baseball_ball, 44.6ms
video 1/1 (frame 3/341) /Users/kwonsebin/Documents/khu/4-1/capstone/server/uploads/8.mp4: 384x640 1 Baseball_ball, 38.1ms
video 1/1 (frame 4/341) /Users/kwonsebin/Documents/khu/4-1/capstone/server/uploads/8.mp4: 

In [20]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False)

# 저장 폴더 생성
output_folder = "uploads"
os.makedirs(output_folder, exist_ok=True)

UPLOAD_FOLDER = 'xxxxx'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

I0000 00:00:1733156271.599016 11448648 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro


W0000 00:00:1733156271.753970 11484887 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733156271.829739 11484891 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [22]:



R0x1, R0y1, R0x2, R0y2, R3x1, R3y1, R3x2, R3y2 = average(detection_results3)
            # print(R0x1, R0y1, R0x2, R0y2, R3x1, R3y1, R3x2, R3y2)
result_dict = {}

# for (i, result) in enumerate(detection_results3):
result = detection_results3[30]
i = 30
i_dict = {}
img = result.orig_img  # YOLO 결과 원본 이미지
annotated_image = img.copy()
print(i, result.boxes.cls)
num_base = sum(1 for obj in result.boxes if obj.cls == 1.)
Two_base = False
Two_base_first = True
if num_base >1:
    Two_base = True
num_glove = sum(1 for obj in result.boxes if obj.cls == 8.)
Two_glove = False
Two_glove_first = True
if num_glove >1:
    Two_glove = True
# num_player = sum(1 for obj in result.boxes if obj.cls == 0. or obj.cls == 3.)
num_Runner = sum(1 for obj in result.boxes if obj.cls == 0.)
num_Baseman = sum(1 for obj in result.boxes if obj.cls == 3.)
over_1_runner = False
cos_list = []
obj_list = []
if num_Runner >1 and num_Baseman >= 1:
    over_1_runner = True
    for obj in result.boxes:
        x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
        
        vec = np.array([x1, y1, x2, y2])
        vec0B = np.array([R0x1, R0y1, R0x2, R0y2])
        cos0 = cosx(vec, vec0B)
        cos_list.append(cos0)
        obj_list.append(obj)
    max_cos_index = cos_list.index(max(cos_list))
    obj = obj_list[max_cos_index]





count = 0
for obj in (result.boxes):
    if over_1_runner:
        x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
        vec = np.array([x1, y1, x2, y2])
        vec0B = np.array([R0x1, R0y1, R0x2, R0y2])
        cos0 = cosx(vec, vec0B)
        if cos0 != max(cos_list):
            continue
        # ----------------------------------

    if obj.cls == 0. or obj.cls == 3.:
        x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
        vec = np.array([x1, y1, x2, y2])
        vec0B = np.array([R0x1, R0y1, R0x2, R0y2])
        vec3B = np.array([R3x1, R3y1, R3x2, R3y2])
        cos0 = dot(vec, vec0B)/(norm(vec)*norm(vec0B))
        cos3 = dot(vec, vec3B)/(norm(vec)*norm(vec3B))

        if cos0 < cos3  : 
        # if obj.cls == 3.:
            cv2.putText(annotated_image, "Baseman", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cropped_person_image = img[y1:y2, x1:x2]

            i_dict[count]=(3, (x1, y1, x2, y2))
            count += 1
            # cv2.imwrite(f"pose_results/cropped_person_image{i}.jpg", cropped_person_image)
            cropped_rgb = cv2.cvtColor(cropped_person_image, cv2.COLOR_BGR2RGB)
            result_pose = pose.process(cropped_rgb)
            # MediaPipe의 포즈 추정 결과를 이미지에 그리기
            mp.solutions.drawing_utils.draw_landmarks(
                annotated_image[y1:y2, x1:x2], result_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        elif cos0 > cos3 :
        # if obj.cls == 0.:  # 클래스가 0인 경우에만 실행
            
            
            x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
            
            
            vecOR = np.array([x1, y1, x2, y2])
            vecRB = np.array([R0x1, R0y1, R0x2, R0y2])
            cos = dot(vecOR, vecRB)/(norm(vecOR)*norm(vecRB))
            if cos > 0.99:
                cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (255, 255, 255), 2)
            else:
                print("cos : ", cos)
                cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (255,255,255), 2)
                # cv2.putText(annotated_image, "correct", (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            i_dict[count]=(0, (x1, y1, x2, y2))
            count += 1
            
            cropped_person_image = img[y1:y2, x1:x2]
            # cv2.imwrite(f"pose_results/cropped_lg{i}.jpg", cropped_person_image)

            cropped_rgb = cv2.cvtColor(cropped_person_image, cv2.COLOR_BGR2RGB)
            result_pose = pose.process(cropped_rgb)

            if result_pose.pose_landmarks:
                landmark_27 = result_pose.pose_landmarks.landmark[27]
                landmark_29 = result_pose.pose_landmarks.landmark[29]
                landmark_31 = result_pose.pose_landmarks.landmark[31]
                landmark_28 = result_pose.pose_landmarks.landmark[28]
                landmark_30 = result_pose.pose_landmarks.landmark[30]
                landmark_32 = result_pose.pose_landmarks.landmark[32]

                x27, y27 = int(landmark_27.x * (x2 - x1)) + x1, int(landmark_27.y * (y2 - y1)) + y1
                x29, y29 = int(landmark_29.x * (x2 - x1)) + x1, int(landmark_29.y * (y2 - y1)) + y1
                x31, y31 = int(landmark_31.x * (x2 - x1)) + x1, int(landmark_31.y * (y2 - y1)) + y1
                x28, y28 = int(landmark_28.x * (x2 - x1)) + x1, int(landmark_28.y * (y2 - y1)) + y1
                x30, y30 = int(landmark_30.x * (x2 - x1)) + x1, int(landmark_30.y * (y2 - y1)) + y1
                x32, y32 = int(landmark_32.x * (x2 - x1)) + x1, int(landmark_32.y * (y2 - y1)) + y1
                
                
                cv2.line(annotated_image, (x27, y27), (x29, y29), (0, 255, 0), 2)
                cv2.line(annotated_image, (x29, y29), (x31, y31), (0, 255, 0), 2)
                cv2.line(annotated_image, (x31, y31), (x27, y27), (0, 255, 0), 2)

                cv2.line(annotated_image, (x28, y28), (x30, y30), (0, 255, 0), 2)
                cv2.line(annotated_image, (x30, y30), (x32, y32), (0, 255, 0), 2)
                cv2.line(annotated_image, (x32, y32), (x28, y28), (0, 255, 0), 2)
                
                for idx in range(27, 33):
                    landmark = result_pose.pose_landmarks.landmark[idx]
                    x, y = int(landmark.x * (x2 - x1)) + x1, int(landmark.y * (y2 - y1)) + y1
                    cv2.circle(annotated_image, (x, y), 5, (0, 255, 0), -1)


                triangle1 = [(x27, y27), (x29, y29), (x31, y31)]
                triangle2 = [(x28, y28), (x30, y30), (x32, y32)]


    
    # if (obj.cls == 8.) or (obj.cls == 3.) or (obj.cls == 2.) or (obj.cls == 1.) or (obj.cls == 0.):
        # if Two_glove and Two_glove_first:
        #     # pass
        #     Bx1, By1, Bx2, By2 = map(int, obj.xyxy[0].tolist())
        #     Two_base_first = False
        # elif Two_glove and not Two_glove_first:
        #     Ax1, Ay1, Ax2, Ay2 = Bx1, By1, Bx2, By2
        #     Bx1, By1, Bx2, By2 = map(int, obj.xyxy[0].tolist())
        #     # 두 개의 사각형을 포함하는 사각형 계산
        #     Cx1, Cy1 = min(Ax1, Bx1), min(Ay1, By1)
        #     Cx2, Cy2 = max(Ax2, Bx2), max(Ay2, By2)
        #     cv2.rectangle(annotated_image, (Cx1, Cy1), (Cx2, Cy2), (0, 255, 255), 2)

        # elif not Two_glove:
        #     Bx1, By1, Bx2, By2 = map(int, obj.xyxy[0].tolist())
        #     cv2.rectangle(annotated_image, (Bx1, By1), (Bx2, By2), (0, 255, 0), 2)

        # center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
        # cv2.putText(annotated_image, f"{obj.cls.item():.0f}", (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
        # cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    if obj.cls == 8.:
        x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
        cv2.putText(annotated_image, "Glove", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 0, 0), 2)
    
    
    if obj.cls == 1.:
        if Two_base and Two_base_first:
            # pass
            Bx1, By1, Bx2, By2 = map(int, obj.xyxy[0].tolist())
            Two_base_first = False
        elif Two_base and not Two_base_first:
            Ax1, Ay1, Ax2, Ay2 = Bx1, By1, Bx2, By2
            Bx1, By1, Bx2, By2 = map(int, obj.xyxy[0].tolist())
            # 두 개의 사각형을 포함하는 사각형 계산
            Bx1, By1 = min(Ax1, Bx1), min(Ay1, By1)
            Bx2, By2 = max(Ax2, Bx2), max(Ay2, By2)
            cv2.putText(annotated_image, "base", (Bx1, By1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(annotated_image, (Bx1, By1), (Bx2, By2), (0, 255, 255), 2)

        elif not Two_base:
            Bx1, By1, Bx2, By2 = map(int, obj.xyxy[0].tolist())
            cv2.putText(annotated_image, "Base", (Bx1, By1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(annotated_image, (Bx1, By1), (Bx2, By2), (0, 255, 255), 2)


    

result_dict[i] = i_dict
            # 삼각형 선분이 직사각형 경계와 교차하는지 확인
if is_triangle_line_in_box(triangle1, (Bx1, By1, Bx2, By2)) or is_triangle_line_in_box(triangle2, (Bx1, By1, Bx2, By2)):
    print("베이스를 밟았습니다")
    cv2.putText(annotated_image, "On base", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)


frame_image_path = os.path.join(UPLOAD_FOLDER, 'result_frame.jpg')
cv2.imwrite(frame_image_path, annotated_image)

30 tensor([6., 0., 1., 3.])


True